In [38]:
from catboost import Pool, CatBoostClassifier
from collections import defaultdict
from pymystem3 import Mystem
from os import walk

In [212]:
m = Mystem()
m.analyze('красивый, «Ведомости», ыва')

[{'analysis': [{'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)',
    'lex': 'красивый',
    'wt': 1}],
  'text': 'красивый'},
 {'text': ', «'},
 {'analysis': [{'gr': 'S,жен,неод=(пр,ед|вин,мн|дат,ед|род,ед|им,мн)',
    'lex': 'ведомость',
    'wt': 1}],
  'text': 'Ведомости'},
 {'text': '», '},
 {'analysis': [{'gr': 'S,муж,неод=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)',
    'lex': 'ыва',
    'qual': 'bastard',
    'wt': 0.2000164368}],
  'text': 'ыва'},
 {'text': '\n'}]

In [57]:
def readTrain():
    (_, _, filenames) = next(walk('Collection3'))
    lines = []
    for name in filenames:
        if 'ann' in name:
            with open('Collection3/' + name) as f:
                lines += [line for line in f.readlines()]
    return lines

In [58]:
def readRaw():
    (_, _, filenames) = next(walk('Collection3'))
    lines = []
    for name in filenames:
        if 'ann' not in name:
            with open('Collection3/' + name) as f:
                lines += [line for line in f.readlines()]
    return lines

In [247]:
import gensim

w2v_fpath = "all.norm-sz100-w10-cb0-it1-min100.w2v"
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')

In [337]:
def toW2vTag(tag):
    if tag == 'S':
        return 'NOUN'
    if tag == 'A':
        return 'ADJ'
    return tag

def analyzeWord(word, isUsed):
    features = []
    word = re.sub('["\'«»<>]', '"', word)
    raw = m.analyze(word)[0]['analysis'][0]
    tag = raw['gr'][0]
    isNoun = tag == 'S'
    features.append(isNoun)
    isAdj = tag == 'A'
    features.append(isAdj)
    isAnother = not isNoun and not isAdj
    features.append(isAnother)
    isName = 'имя' in ['gr'] or 'фам' in raw['gr']
    features.append(isName)
    isCapital = sum([char.isupper() for char in word])
    features.append(isCapital)
    isInBrackets = word[0] == '"' or word[-1] == '"'
    features.append(isInBrackets)
    w2v_name = 0
    w2v_surname = 0
    w2v_org = 0
    w2v_person = 0
    w2v_company = 0
    w2v_tag = toW2vTag(tag)
    w2v_word =  raw['lex'].lower()
    try: w2v_surname = w2v.similarity('фамилия', w2v_word)
    except Exception as e:
        pass
    try: w2v_name = w2v.similarity('имя', w2v_word)
    except: pass
    try: w2v_org = w2v.similarity('организация', w2v_word)
    except: pass
    try: w2v_person = w2v.similarity('человек', w2v_word)
    except: pass
    try: w2v_company = w2v.similarity('компания', w2v_word)
    except: pass
    features.append(w2v_name)
    features.append(w2v_surname)
    features.append(w2v_org)
    features.append(w2v_person)
    features.append(w2v_company)
    return features

In [338]:
import numpy as np

def analyzeSentence(sentence, isUsed):
    features = []
    for word in sentence.split():
        try:
            if np.random.random() < 0.2:
                features.append(analyzeWord(word, isUsed))
        except:
            continue
    return features

In [349]:
from sklearn.model_selection import train_test_split

train_data = []
train_label = []
isUsed = defaultdict(bool)
m = Mystem()

for line in readTrain():
    label = line.split()[1]
    for word in line.split()[4:]:
        try: 
            train_data.append(analyzeWord(word, isUsed))
            train_label.append(label)
        except Exception as e:
            continue
    
for line in readRaw():
    label = 'NONE'
    features = analyzeSentence(line, isUsed)
    train_data += features
    train_label += [label for word in features]

cat_features = [0]

print(train_data)
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.33, random_state=42)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



NameError: name 'eval_dataset' is not defined

In [ ]:
train_dataset = Pool(data=X_train,
                     label=y_train)
                     #cat_features=cat_features)
    
test_dataset = Pool(data=X_test,
                     label=y_test)
                     #cat_features=cat_features)
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=5000,
                           depth=11,
                           loss_function='MultiClass')
# Fit model with `use_best_model=True`

model.fit(train_dataset,
          eval_set=test_dataset,
          use_best_model=True)

print("Count of trees in model = {}".format(model.tree_count_))

0:	learn: 1.3396129	test: 1.3397950	best: 1.3397950 (0)	total: 353ms	remaining: 29m 24s
1:	learn: 1.2958351	test: 1.2964447	best: 1.2964447 (1)	total: 428ms	remaining: 17m 49s
2:	learn: 1.2582201	test: 1.2590762	best: 1.2590762 (2)	total: 511ms	remaining: 14m 11s
3:	learn: 1.2222710	test: 1.2235439	best: 1.2235439 (3)	total: 598ms	remaining: 12m 27s
4:	learn: 1.1879577	test: 1.1895709	best: 1.1895709 (4)	total: 696ms	remaining: 11m 35s
5:	learn: 1.1566806	test: 1.1585797	best: 1.1585797 (5)	total: 790ms	remaining: 10m 57s
6:	learn: 1.1280336	test: 1.1302496	best: 1.1302496 (6)	total: 886ms	remaining: 10m 31s
7:	learn: 1.1011097	test: 1.1036855	best: 1.1036855 (7)	total: 992ms	remaining: 10m 18s
8:	learn: 1.0765492	test: 1.0793006	best: 1.0793006 (8)	total: 1.11s	remaining: 10m 14s
9:	learn: 1.0534275	test: 1.0563458	best: 1.0563458 (9)	total: 1.21s	remaining: 10m 5s
10:	learn: 1.0314122	test: 1.0345651	best: 1.0345651 (10)	total: 1.32s	remaining: 9m 57s
11:	learn: 1.0101820	test: 1.013

93:	learn: 0.5342640	test: 0.5482981	best: 0.5482981 (93)	total: 9.44s	remaining: 8m 12s
94:	learn: 0.5329784	test: 0.5470430	best: 0.5470430 (94)	total: 9.53s	remaining: 8m 12s
95:	learn: 0.5316542	test: 0.5457774	best: 0.5457774 (95)	total: 9.63s	remaining: 8m 11s
96:	learn: 0.5302819	test: 0.5444599	best: 0.5444599 (96)	total: 9.72s	remaining: 8m 11s
97:	learn: 0.5290232	test: 0.5433129	best: 0.5433129 (97)	total: 9.82s	remaining: 8m 11s
98:	learn: 0.5279616	test: 0.5423477	best: 0.5423477 (98)	total: 9.93s	remaining: 8m 11s
99:	learn: 0.5265610	test: 0.5411151	best: 0.5411151 (99)	total: 10s	remaining: 8m 11s
100:	learn: 0.5253601	test: 0.5400385	best: 0.5400385 (100)	total: 10.1s	remaining: 8m 10s
101:	learn: 0.5242069	test: 0.5389881	best: 0.5389881 (101)	total: 10.2s	remaining: 8m 10s
102:	learn: 0.5231328	test: 0.5379768	best: 0.5379768 (102)	total: 10.3s	remaining: 8m 10s
103:	learn: 0.5220267	test: 0.5369249	best: 0.5369249 (103)	total: 10.4s	remaining: 8m 10s
104:	learn: 0.5

187:	learn: 0.4682587	test: 0.4898098	best: 0.4898098 (187)	total: 18.5s	remaining: 7m 54s
188:	learn: 0.4678995	test: 0.4895034	best: 0.4895034 (188)	total: 18.6s	remaining: 7m 53s
189:	learn: 0.4675410	test: 0.4891949	best: 0.4891949 (189)	total: 18.7s	remaining: 7m 53s
190:	learn: 0.4671674	test: 0.4888963	best: 0.4888963 (190)	total: 18.8s	remaining: 7m 53s
191:	learn: 0.4666814	test: 0.4885503	best: 0.4885503 (191)	total: 18.9s	remaining: 7m 53s
192:	learn: 0.4661892	test: 0.4881253	best: 0.4881253 (192)	total: 19s	remaining: 7m 52s
193:	learn: 0.4658497	test: 0.4878690	best: 0.4878690 (193)	total: 19.1s	remaining: 7m 52s
194:	learn: 0.4653980	test: 0.4874910	best: 0.4874910 (194)	total: 19.1s	remaining: 7m 51s
195:	learn: 0.4649571	test: 0.4871395	best: 0.4871395 (195)	total: 19.2s	remaining: 7m 51s
196:	learn: 0.4645843	test: 0.4868547	best: 0.4868547 (196)	total: 19.3s	remaining: 7m 51s
197:	learn: 0.4642695	test: 0.4866119	best: 0.4866119 (197)	total: 19.4s	remaining: 7m 51s
1

279:	learn: 0.4426157	test: 0.4699302	best: 0.4699302 (279)	total: 26.9s	remaining: 7m 34s
280:	learn: 0.4424823	test: 0.4698435	best: 0.4698435 (280)	total: 27s	remaining: 7m 33s
281:	learn: 0.4423534	test: 0.4697630	best: 0.4697630 (281)	total: 27.1s	remaining: 7m 33s
282:	learn: 0.4421648	test: 0.4696209	best: 0.4696209 (282)	total: 27.2s	remaining: 7m 33s
283:	learn: 0.4419886	test: 0.4694866	best: 0.4694866 (283)	total: 27.3s	remaining: 7m 33s
284:	learn: 0.4418298	test: 0.4693769	best: 0.4693769 (284)	total: 27.4s	remaining: 7m 33s
285:	learn: 0.4415618	test: 0.4691597	best: 0.4691597 (285)	total: 27.5s	remaining: 7m 32s
286:	learn: 0.4413231	test: 0.4689797	best: 0.4689797 (286)	total: 27.6s	remaining: 7m 32s
287:	learn: 0.4411017	test: 0.4688041	best: 0.4688041 (287)	total: 27.7s	remaining: 7m 32s
288:	learn: 0.4409406	test: 0.4686831	best: 0.4686831 (288)	total: 27.7s	remaining: 7m 32s
289:	learn: 0.4409092	test: 0.4686645	best: 0.4686645 (289)	total: 27.8s	remaining: 7m 31s
2

370:	learn: 0.4265110	test: 0.4587486	best: 0.4587486 (370)	total: 35.3s	remaining: 7m 20s
371:	learn: 0.4263813	test: 0.4586756	best: 0.4586756 (371)	total: 35.4s	remaining: 7m 20s
372:	learn: 0.4262309	test: 0.4585589	best: 0.4585589 (372)	total: 35.5s	remaining: 7m 20s
373:	learn: 0.4260750	test: 0.4584733	best: 0.4584733 (373)	total: 35.6s	remaining: 7m 20s
374:	learn: 0.4258795	test: 0.4583633	best: 0.4583633 (374)	total: 35.7s	remaining: 7m 20s
375:	learn: 0.4256887	test: 0.4582589	best: 0.4582589 (375)	total: 35.8s	remaining: 7m 19s
376:	learn: 0.4255442	test: 0.4581718	best: 0.4581718 (376)	total: 35.9s	remaining: 7m 19s
377:	learn: 0.4253640	test: 0.4580320	best: 0.4580320 (377)	total: 36s	remaining: 7m 19s
378:	learn: 0.4251450	test: 0.4578991	best: 0.4578991 (378)	total: 36.1s	remaining: 7m 19s
379:	learn: 0.4250051	test: 0.4578018	best: 0.4578018 (379)	total: 36.1s	remaining: 7m 19s
380:	learn: 0.4248390	test: 0.4577055	best: 0.4577055 (380)	total: 36.2s	remaining: 7m 19s
3

In [273]:
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

In [ ]:
import re

def processLine(line):
    line = re.sub('["\'«»]', '"', line)
    inds = findOccurrences(line, '"')
    ans = ""
    for i in range(0, len(inds), 2):
        if i + 1 == len(inds):
            break
        ans += str(inds[i] + 1) + ' ' + str(inds[i + 1] - inds[i] - 1) + ' ORG '
    for raw in m.analyze(line):
        featWord = []
        try:
            if 'имя' in raw['analysis'][0]['gr'] or 'фам' in raw['analysis'][0]['gr']:
                ans += str(line.find(raw['text']) - 1) + ' ' + str(len(raw['text'])) + ' PERSON '
        except:
            continue
    with open('out.txt', 'a') as f:
        f.write(ans + 'EOL\n')

In [340]:
def processLine(line):
    words = line.split()
    features = []
    ans = ""
    if len(line) < 2:
        print('OK')
        return
    for word in words:
        try:
            word = word.replace(',', '').replace('.', '').replace('?', '').replace('!', '').replace('\n', '')
            pred_class = model.predict(analyzeWord(word, isUsed))
            if pred_class[0] == 'ORG':
                pos = line.find(word)
                ans += str(pos) + ' ' + str(len(word)) + ' ORG '
            if pred_class[0] == 'PER':
                pos = line.find(word)
                ans += str(pos) + ' ' + str(len(word)) + ' PERSON  '  
        except Exception as e:
            #print(e)
            pass
    with open('out.txt', 'a') as f:
        f.write(ans + 'EOL\n')

In [341]:
def processTest():
    with open('dataset_40163_1.txt') as f:
        [processLine(line) for line in f.read().split('\n')]

In [356]:
import os

m = Mystem()
try:
    os.remove('out.txt')
except: pass
processTest()

OK
